In [ ]:
import os
import pickle
import subprocess
from typing import Tuple
import wikipedia
import numpy as np
import pandas as pd
import os.path
from os import path
import nltk

In [ ]:
def actor_information(actor_name):
    actor_page = wikipedia.page(actor_name)
    #print(actor_name.section('Public image'))
    #print(actor_page.title)
    #print(actor_page.url)
    #print(actor_page.content)
    #print(actor_page.summary)
    #print(actor_page.categories)
    return (actor_page.title, actor_page.content)

def nlkt_pull():
    if (path.exists('/root/nltk_data')):
        return
    nltk.download('punkt', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    nltk.download('maxent_ne_chunker', quiet=True)
    nltk.download('words', quiet=True)
    
nlkt_pull()

In [ ]:
#text = actor_information("Bradley Cooper")[1]
text = actor_information("Tom Cruise")[1]
text = actor_information("Bruce Willis")[1]

In [ ]:


sentences = []
sent_tokens = []

for sent in nltk.sent_tokenize(text):
    sentences.append(sent)
    sent_tokens.append(nltk.word_tokenize(sent))

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 0)

pd.DataFrame({"sentence": sentences, "tokens" :sent_tokens})

In [ ]:
from snorkel.labeling import labeling_function
import re
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1
ROLE_CATEGORY_COMEDY = r"\b(comed(y|ian|ies))"

@labeling_function()
def role_category_comedy(x):
    return POSITIVE if re.search(ROLE_CATEGORY_COMEDY, x.sentence.lower()) else ABSTAIN

ROLE_CATEGORY_DRAMA = r"\b(drama(s))"

@labeling_function()
def role_category_drama(x):
    return NEGATIVE if re.search(ROLE_CATEGORY_DRAMA, x.sentence.lower()) else ABSTAIN

In [ ]:
from snorkel.labeling import LabelModel, PandasLFApplier

lfs = [role_category_comedy, role_category_drama]
# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs)
df_train = pd.DataFrame({"sentence": sentences})
L_train = applier.apply(df_train)

In [ ]:
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [ ]:
df_train = df_train[df_train.label != ABSTAIN]

In [ ]:
df_train